In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=40e3f739350c678f849b23bd90f9494eb8e95014e9b00d5b0d3eb2907f1c3974
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator


In [3]:
spark = SparkSession.builder \
    .appName("WineQualityPrediction") \
    .getOrCreate()


In [4]:
training_data = spark.read.csv("/TrainingDataset - TrainingDataset.csv", header=True, inferSchema=True)


In [7]:
feature_cols = training_data.columns[:-1]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="assembled_features")
prepared_data = assembler.transform(training_data)



In [9]:
from pyspark.sql.functions import col


if "assembled_features" in prepared_data.columns:
    prepared_data = prepared_data.withColumnRenamed("assembled_features", "old_assembled_features")


assembler = VectorAssembler(inputCols=feature_cols, outputCol="assembled_features")
prepared_data = assembler.transform(prepared_data)


if "old_assembled_features" in prepared_data.columns:
    prepared_data = prepared_data.drop("old_assembled_features")


In [12]:
from pyspark.ml.regression import LinearRegression


output_col_name = "assembled_features"
while output_col_name in prepared_data.columns:

    output_col_name += "_new"

assembler = VectorAssembler(inputCols=feature_cols, outputCol=output_col_name)
prepared_data = assembler.transform(prepared_data)

lr = LinearRegression(labelCol="quality", featuresCol=output_col_name)
pipeline = Pipeline(stages=[lr])

model = pipeline.fit(prepared_data)



In [18]:
model.save("/Users/doc/Documents/download/trr")


In [14]:
validation_data = spark.read.csv("/ValidationDataset - ValidationDataset.csv", header=True, inferSchema=True)
prepared_validation_data = assembler.transform(validation_data)
predictions = model.transform(prepared_validation_data)
evaluator = RegressionEvaluator(labelCol="quality", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on validation data = %g" % rmse)


Root Mean Squared Error (RMSE) on validation data = 0.689081
